In [51]:
import os
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

print("API Key cargada")

API Key cargada


1. Implementación Práctica de RAG:
Recrea el ejemplo dado en un Jupyter Notebook o entorno de desarrollo similar, utilizando las librerías langchain_community.document_loaders, langchain_text_splitters, langchain_ollama, langchain_chroma, y langchain_google_genai.
Verifica el funcionamiento del sistema RAG respondiendo a preguntas relacionadas con el contenido de un documento X (deben proporcionar uno ustedes).

In [66]:
pdf_path = "Guia-de-apoyo-Presentacion-Trabajo-Final-de-Integracion.pdf"

try:
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()
    print("documento", pdf_path, "cargado. Contiene", len(pages), "páginas.")
except FileNotFoundError:
    print("Error: No se encontró el archivo", pdf_path)
    raise

documento Guia-de-apoyo-Presentacion-Trabajo-Final-de-Integracion.pdf cargado. Contiene 3 páginas.


In [65]:
# dividir el texto en chunks
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=100
)
# procesar la lista de páginas
texts_chunks = text_splitter.split_documents(pages)
print("documento dividido en", len(texts_chunks), "chunks") 

documento dividido en 8 chunks


In [54]:
# modelo local de HuggingFace
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings_model = HuggingFaceEmbeddings(model_name=model_name)

In [55]:
#vector store
vector_store = Chroma.from_documents(
    documents=texts_chunks, 
    embedding=embeddings_model
)

In [56]:
# Configurar LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=api_key,
    temperature=0.3
)

In [57]:
#retriever desde el vector store
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [58]:
#prompt
template = """
Eres un asistente servicial y preciso. Responde la pregunta del usuario basándote
SOLAMENTE en el siguiente contexto. Si la respuesta no está en el contexto,
di "No tengo información sobre eso en el documento".

Contexto:
{context}

Pregunta:
{question}

Respuesta:
"""
prompt = ChatPromptTemplate.from_template(template)

In [59]:
# formatear docs
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [60]:
#cadena rag
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [61]:
question1 = "que me recomiendas para hacer una buena presentacion final de integracion?"
response1 = rag_chain.invoke(question1)
print(response1)

Para hacer una buena presentación final de integración, la guía recomienda organizar y estructurar un Storyboard o guión para la presentación de la propuesta de valor. Es fundamental que el LIENZO DE MODELO DE NEGOCIOS y el WHITEPAPER sean profundamente comprendidos por los integrantes del equipo. Además, deben tener respuestas claras a las siguientes preguntas: ¿Qué problema estamos intentando resolver? y ¿A quiénes afecta el problema y por qué es?


In [62]:
question2 = "cuanto tiempo debe durar la presentación?"
response2 = rag_chain.invoke(question2)
print(response2)

No tengo información sobre eso en el documento.


In [63]:
question3 = "como hago la arquitectura de alto nivel implementada?"
response3 = rag_chain.invoke(question3)
print(response3)


Para realizar la arquitectura de alto nivel implementada, debes hacer un esquema que represente los componentes de la arquitectura y sus interrelaciones. Este esquema debe incluir los lenguajes utilizados, la base de datos, las APIs, las bibliotecas y las interacciones. Se recomienda pedir ayuda a los profesores del módulo Taller de Lenguajes de Programación Avanzado.


2. Exploración de Modelos de Embedding y LLM:
Experimenta con diferentes modelos de embedding disponibles en langchain_ollama (además de "nomic-embed-text") y analiza cómo impactan en la calidad de la recuperación de documentos.
Prueba con otros modelos de lenguaje de gran tamaño (LLM) compatibles con langchain_google_genai o langchain_ollama y compara sus respuestas y rendimiento.

3. Optimización del Separador de Texto:
Modifica los parámetros chunk_size y chunk_overlap del CharacterTextSplitter y observa cómo afectan a la creación de los documentos y, consecuentemente, a la precisión de las respuestas del LLM.
Investiga otros tipos de separadores de texto disponibles en Langchain y evalúa su idoneidad para diferentes tipos de documentos.

4. Gestión de Bases de Datos Vectoriales:
Explora las funcionalidades de Chroma para gestionar colecciones, persistencia de datos y realizar búsquedas más avanzadas (por ejemplo, filtrado de resultados).
Investiga otras bases de datos vectoriales compatibles con Langchain y considera sus ventajas y desventajas para diferentes casos de uso.

5. Refinamiento de Prompts:
Experimenta con diferentes prompt_template para el LLM, ajustando las instrucciones del sistema y los mensajes de IA/humano para mejorar la calidad y relevancia de las respuestas.
Implementa técnicas de ingeniería de prompts para guiar al LLM a proporcionar respuestas más precisas y útiles, incluyendo la atribución de fuentes cuando sea posible.